# 0. Packages and data

In [ ]:
path='G:/Shared drives/BeStarsMiMeS/UpdatedFiles/' #Patrick's google file stream path

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
import specpolFlow as pol

In [ ]:

from matplotlib.backends.backend_pdf import PdfPages

import pandas as pd

In [ ]:
import pandas as pd
sheet_id = '1M6y1Wnsrc-w5FjUMfKaSFa_-foIDAaMe8W4lYNWnWyk'
sheet_name = 'Stars'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
StarData=pd.read_csv(url)

sheet_name = 'Observations'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
Observations=pd.read_csv(url)

In [ ]:
import itertools


### This gets rid of the warnings in the cells where it is creating more than 20 figures (i.e. 1.2)
#### Not needed for this notebook but could be useful in the future
#import warnings
#warnings.filterwarnings("default", message="More than 20 figures have been opened.")


# 1. One Model


In [ ]:
'''
import copy

path='/content/drive/Shareddrives/BeStarsMiMeS/03-LSD-calculations/'
#model='normalized__CleanAsif+tellClean'
model=models[1]
for j, ObsInfo in Observations.iterrows():

  star=Observations['Name'][j]
  #retrieves the vsini for the star
  for i,StarInfo in StarData.iterrows():
    if StarData['Name'][i]==Observations['Name'][j]:
      vsini=StarData['Adopted-vsini'][i]
      break

  file1 = Observations['LSDfileName'][j].split('.s')
  file='{}LSD/{}/{}'.format(path,model,file1[0])
  lsd = pol.iolsd.read_lsd(file)
  Bz=pol.bz.calcBz(lsd=lsd,cog='I',velrange=[-vsini,vsini],plot=False) #calculates the longitudinal field
  if j==0:
    Bz_table=copy.copy(Bz)
  else:
    Bz_table = pd.concat([Bz_table, Bz], ignore_index = True)

# add rows to the table
names = Observations['Name']
Bz_table.insert(0, "Star", names, True)

#saves the table to a .csv file
writefile='/content/drive/Shareddrives/BeStarsMiMeS/04-Bz-calculations/{}.csv'.format(model)
with open(writefile,'w', encoding = 'utf-8-sig') as f:
  Bz_table.to_csv(f)
  '''

# 2. Multi-page PDF

In [ ]:
models=['hybrid_maskdepth0.1_tellClean',
 'hybrid_maskdepth0.02_tellClean',
 'hybrid_CleanAsif+tellClean',
 'hybrid_CleanAsifTweak+tellClean',
 'normalized_maskdepth0.1_tellClean',
 'normalized_maskdepth0.02_tellClean',
 'normalized_CleanAsif+tellClean',
 'normalized_CleanAsifTweak+tellClean']

In [ ]:
%%capture
import copy

#ObservationsShort=Observations[0:20]
for f in range(len(models)):
  model=models[f]
  start=0
  with PdfPages('{}'.format(model)) as pdf:
    for j, ObsInfo in Observations.iterrows():

      star=Observations['Name'][j]
      vrad=Observations['Unnamed: 11'][j]
      
      #retrieves vsini
      for i,StarInfo in StarData.iterrows():
        if StarData['Name'][i]==Observations['Name'][j]:
          if StarData['vsini-estimate'][i]=='very bad' or np.isnan(StarData['vsini-estimate'][i]):
            vsini=StarData['Adopted-vsini'][i]
            break
          else: 
            vsini=StarData['vsini-estimate'][i]
            break

      file1 = Observations['LSDfileName'][j].split('.s')
      file='{}03-LSD-calculations/LSD/{}/{}'.format(path,model,file1[0])
      lsd = pol.iolsd.read_lsd(file)
      Bz,fig=pol.bz.calcBz(lsd=lsd,cog=vrad,velrange=[(vrad-1.5*vsini),(vrad+1.5*vsini)],plot=True, bzwidth=vsini) #calculates longitudinal field
      Bz=pd.DataFrame.from_dict(Bz,'index').T
      name = Observations['Name'][j]
      Bz.insert(0,"Star", name, True)
      if start==0:
        Bz_table=copy.copy(Bz)
        start=1
      else:
        Bz_table = pd.concat([Bz_table, Bz], ignore_index = True)
      fig.suptitle('{}'.format(file1[0]),fontsize=20,y=0.92)
      pdf.savefig(fig)

    writefile='{}04-Bz-calculations/{}.csv'.format(path,model)
    with open(writefile,'w', encoding = 'utf-8-sig') as f:
      Bz_table.to_csv(f)



# 3. Well Behaved Stars

In [ ]:
import pandas as pd
sheet_id = '1M6y1Wnsrc-w5FjUMfKaSFa_-foIDAaMe8W4lYNWnWyk'
sheet_name = 'WellBehavedStars'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
StarData=pd.read_csv(url)

sheet_name = 'Observations'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
Observations=pd.read_csv(url)

In [ ]:
%%capture
import copy

#ObservationsShort=Observations[0:20]
for f in range(len(models)):
  model=models[f]
  start=0
  with PdfPages('{}04-Bz-calculations/WellBehavedStars/{}_GOOD'.format(path,model)) as pdf:
    for i, StarInfo in StarData.iterrows():
      star=StarData['Name'][i]
      if StarData['vsini-estimate'][i]=='very bad':
        vsini=StarData['Adopted-vsini'][i]
      else:
        vsini=StarData['vsini-estimate'][i]
      for j, ObsInfo in Observations.iterrows():
        if Observations['Name'][j]==star:
          vrad=Observations['Unnamed: 11'][j]          
          #retrieves vsini
          file1 = Observations['LSDfileName'][j].split('.s')
          file='{}03-LSD-calculations/LSD/{}/{}'.format(path,model,file1[0])
          lsd = pol.iolsd.read_lsd(file)
          Bz,fig=pol.bz.calcBz(lsd=lsd,cog=vrad,velrange=[(vrad-1.5*vsini),(vrad+1.5*vsini)],plot=True, bzwidth=vsini) #calculates longitudinal field
          Bz=pd.DataFrame.from_dict(Bz,'index').T
          name = Observations['Name'][j]
          Bz.insert(0,"Star", name, True)
          if start==0:
            Bz_table=copy.copy(Bz)
            start=1
          else:
            Bz_table = pd.concat([Bz_table, Bz], ignore_index = True)
          fig.suptitle('{}'.format(file1[0]),fontsize=20,y=0.92)
          pdf.savefig(fig)
        else:
          continue

    writefile='{}04-Bz-calculations/WellBehavedStars/{}_GOOD.csv'.format(path,model)
    with open(writefile,'w', encoding = 'utf-8-sig') as f:
      Bz_table.to_csv(f)

      

